In [32]:
from bs4 import BeautifulSoup
from itertools import cycle
from requests import get

In [30]:
class RotateProxies:
    def __init__(self):
        self.proxies = self.collectProxies()

    def collectProxies(self):
        url = "https://free-proxy-list.net/"
        table = BeautifulSoup(get(url).text, 'html.parser').find(
            'table', attrs={'id': 'proxylisttable'})

        proxies = cycle([{
            "https":
            "{}:{}".format(
                row.find_all('td')[0].string,
                row.find_all('td')[1].string)
        } for row in table.find_all('tr') if len(row.find_all('td')) > 1
                         and row.find_all('td')[6].string == "yes"])

        return proxies
    
    def next(self):
        test_url = 'https://httpbin.org/ip'
        proxy = None
        while True:
            proxy = next(self.proxies)
            try:
                test = get(test_url, proxies = proxy)
            except:
                continue
            else:
                break
        return proxy

In [33]:
p = RotateProxies()

In [35]:
p.next()

{'https': '83.13.164.202:49185'}